In [1]:
import pandas as pd
import requests
import json
import string 
import numpy as np
import os
from tqdm import tqdm

In [4]:
with open('s2_ids.json','r') as infile:
    data = json.load(infile)

In [5]:
paper_id_list = data['s2_id_list']

In [7]:
fields = ['title', 'year', 'abstract', 'citations.paperId', 'references.paperId', 'citationCount']
paper_fields = '?fields=' + ','.join(fields)
paper_url = 'https://api.semanticscholar.org/graph/v1/paper/{}'
header = {'x-api-key': api_key} # S2 provide API keys for increased limits, using the API without key is possible
# identical to querying by DOI 
index_list = list(paper_id_list)
batch_size = 20
num_batches = int(len(paper_id_list)/batch_size)
meta_data = []
for batch_num in tqdm(range(num_batches)):
    query = {"ids": index_list[batch_num*batch_size:(batch_num+1)*batch_size]}
    url = paper_url.format('batch') + paper_fields
    r = requests.post(url, json = query)
    res = json.loads(r.text)
    for i in range(batch_size):
        try:
            if res[i]:
                meta_data.append(res[i])
        except: 
            None 

100%|███████████████████████████████████████████████████████████| 3/3 [00:34<00:00, 11.37s/it]


In [ ]:
# repeat for final batch
if len(doi_list)%batch_size != 0:
    index = index_list[(batch_num+1)*batch_size:len(doi_list)]
    query = {"ids": doi_list[(batch_num+1)*batch_size:len(doi_list)]}
    url = paper_url.format('batch') + paper_fields
    r = requests.post(url, json = query, headers = header)
    res = json.loads(r.text)
    for i in range(len(res)):
        if res[i]:
            res[i].update({'original_index':index[i]})
            meta_data.append(res[i])

In [11]:
df = pd.DataFrame(meta_data)

""


In [3]:
df.sample()

,paperId,title,abstract,fieldsOfStudy,s2FieldsOfStudy,embedding,s2FOS,scibert_cls,title_lang,abstract_lang,paperId_add,title_add,year,citationCount,citations,references
5883,daf4cc76c355d19d7d392be50bf97aa4d20bc9f8,Circulating long non-coding RNAs in plasma of ...,BACKGROUND\nWe examined the levels of long non...,"[Biology, Medicine]","[{'category': 'Biology', 'source': 'external'}...","{'model': 'specter@v0.1.1', 'vector': [-0.6012...","[Biology, Medicine, Medicine, Biology]","[-0.5647488236, 0.4838282466, -0.2315677404, -...",en,en,daf4cc76c355d19d7d392be50bf97aa4d20bc9f8,Circulating long non-coding RNAs in plasma of ...,2013.0,249,"[45c51f0cf56b70176fe9e96f83b7a9ca0a2ce3f4, 658...","[92978bb85c91f00979b3ed86c7e4ebfd62c58c21, 2ec..."


In [5]:
paper_ids = set(df['paperId'])
final_df.loc[:,'in_set_refs'] = df['references'].map(lambda x: set(x).intersection(paper_ids))
final_df.loc[:,'in_set_cits'] = dfj['citations'].map(lambda x: set(x).intersection(paper_ids))

In [7]:
final_df.loc[:,'adjacencies'] = df.apply(lambda row:
                                              row['in_set_refs'].union(row['in_set_cits']),
                                              axis = 1)
final_df.to_json('meta_data.jsonl', orient = 'records', lines = True)

In [26]:
G_total = nx.Graph()
for n1, data in df.set_index('paperId').iterrows():
    G_total.add_node(n1, year = data['year'])
for n1, data in df.set_index('paperId').iterrows():
    for n2 in data['adjacencies']:
        G_total.add_edge(n1, n2)

In [27]:
G_total.number_of_nodes(), G_total.number_of_edges()

(58304, 856153)

In [42]:
subgraphs = {}
for y in reversed(range(2014,2022)):
    subgraph = G_total.subgraph(
        [node for node, year_attr in G_total.nodes(data='year')
         if year_attr <= y])
    subgraphs[y] = subgraph

2021
2020
2019
2018
2017
2016
2015
2014


In [52]:
for y in subgraphs:
    nx.write_gexf(subgraphs[y],'citation_graphs/{}.gexf'.format(y))